In [1]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
df = pd.read_csv("R_data/NO2_final.csv", parse_dates=["date"]).set_index("date")

In [3]:
df.head()

,nox,no2,site,code,latitude,longitude,site_type
date,,,,,,,
1996-08-08 00:00:00,43.93,43.93,"- National Physical Laboratory, Teddington",TD0,51.424304,-0.345715,Suburban
1996-08-08 01:00:00,55.39,53.48,"- National Physical Laboratory, Teddington",TD0,51.424304,-0.345715,Suburban
1996-08-08 02:00:00,45.84,45.84,"- National Physical Laboratory, Teddington",TD0,51.424304,-0.345715,Suburban
1996-08-08 03:00:00,36.29,34.38,"- National Physical Laboratory, Teddington",TD0,51.424304,-0.345715,Suburban
1996-08-08 04:00:00,34.38,32.47,"- National Physical Laboratory, Teddington",TD0,51.424304,-0.345715,Suburban


In [ ]:
daily_df = df.groupby(by=["code"]).resample("D").mean().reset_index()
daily_df["date"] = daily_df["date"].apply(lambda x: x.strftime("%Y-%m-%d"))
daily_df.set_index("date", inplace=True)

In [ ]:
daily_df.head()

In [6]:
monthly_df = df.groupby(by=["code"]).resample("M", convention="start").mean().reset_index()
monthly_df["date"] = monthly_df["date"].apply(lambda x: x.strftime("%Y-%m"))
monthly_df.set_index("date", inplace=True)

In [7]:
monthly_df.head()

,code,nox,no2,latitude,longitude
date,,,,,
1997-03,A30,247.941875,60.560430,51.373553,-0.29197
1997-04,A30,263.677406,71.724523,51.373553,-0.29197
1997-05,A30,251.253255,69.521366,51.373553,-0.29197
1997-06,A30,240.406770,61.292412,51.373553,-0.29197
1997-07,A30,257.308118,54.153329,51.373553,-0.29197


In [8]:
yearly_df = df.groupby(by=["code"]).resample("Y").mean().reset_index()
yearly_df["date"] = yearly_df["date"].apply(lambda x: x.strftime("%Y"))
yearly_df.set_index("date", inplace=True)

In [9]:
yearly_df.head()

,code,nox,no2,latitude,longitude
date,,,,,
1997,A30,314.269389,65.982131,51.373553,-0.29197
1998,A30,291.840156,56.801657,51.373553,-0.29197
1999,A30,256.120856,58.470782,51.373553,-0.29197
2000,A30,212.220329,54.903419,51.373553,-0.29197
2001,A30,184.025497,53.486050,51.373553,-0.29197


In [4]:
import gpflow
from gpflow.utilities import print_summary
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
import numpy as np

In [5]:
# Currently, this causes nan values in the df; need to investigate why
gp_df = df.groupby(by=["code"]).resample("M", convention="start").mean().dropna().reset_index()

In [6]:
gp_df = gp_df.loc[gp_df["date"] >= "2000"]

In [7]:
n_months = 12
start_year = gp_df["date"].min().year
gp_df['t'] = gp_df.apply(lambda row: (row.date.year-start_year)*n_months + (row.date.month%n_months), axis=1)

In [8]:
feature_scaler = StandardScaler()
X = gp_df[["latitude", "longitude", "t"]].values
#X[:, 0:2] = feature_scaler.fit_transform(X[:, 0:2] + np.random.normal(0, 1, size=X[:, 0:2].shape))
X[:, 0:2] = feature_scaler.fit_transform(X[:, 0:2])
Y = gp_df[["no2"]].values
Y = feature_scaler.fit_transform(Y)

In [9]:
na = gp_df[gp_df.isna().any(axis=1)]

In [10]:
na.size

0

In [11]:
N, M = X.shape

In [12]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25 )
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(17009, 3) (17009, 1) (5670, 3) (5670, 1)


In [13]:
class SafeMatern52(gpflow.kernels.Matern52):
    def euclid_dist(self, X, X2):
        r2 = self.square_dist(X, X2)
        return tf.sqrt(r2 + 1e-1)

In [ ]:
## KERNEL OPTIONS

k = SafeMatern52(lengthscales=1, active_dims=[0, 1]) + SafeMatern52(lengthscales=0.1, active_dims=[2])
#, lengthscales=[0.01, 0.01, 0.01])
#k = gpflow.kernels.White() + gpflow.kernels.Periodic(gpflow.kernels.IsotropicStationary(), period=12)
#k = gpflow.kernels.Matern52(active_dims=[2]) + gpflow.kernels.Matern52(active_dims=[0, 1])
#k = SafeMatern52(active_dims=[2]) + SafeMatern52(active_dims=[0,1]) + gpflow.kernels.White(3)
#k = gpflow.kernels.White(3) +\
#            gpflow.kernels.RBF(2, active_dims=[0,1], lengthscales=1.0) +\
#            gpflow.kernels.RBF(1, active_dims=[2], lengthscales=1.0) +\
#            gpflow.kernels.Periodic(gpflow.kernels.RBF(1, active_dims=[2], lengthscales=1.0), period=12.0)
#k = gpflow.kernels.White(1)
#k = gpflow.kernels.White() + gpflow.kernels.Matern52(active_dims=[0]) + gpflow.kernels.Matern52(active_dims=[1]) \
#+ gpflow.kernels.Matern52(active_dims=[2])
#k = gpflow.kernels.SquaredExponential()

In [15]:
print_summary(k)

╒═════════════════════════════╤═══════════╤═════════════╤═════════╤═════════════╤═════════╤═════════╤═════════╕
│ name                        │ class     │ transform   │ prior   │ trainable   │ shape   │ dtype   │   value │
╞═════════════════════════════╪═══════════╪═════════════╪═════════╪═════════════╪═════════╪═════════╪═════════╡
│ Sum.kernels[0].variance     │ Parameter │ Softplus    │         │ True        │ ()      │ float64 │     1   │
├─────────────────────────────┼───────────┼─────────────┼─────────┼─────────────┼─────────┼─────────┼─────────┤
│ Sum.kernels[0].lengthscales │ Parameter │ Softplus    │         │ True        │ ()      │ float64 │     1   │
├─────────────────────────────┼───────────┼─────────────┼─────────┼─────────────┼─────────┼─────────┼─────────┤
│ Sum.kernels[1].variance     │ Parameter │ Softplus    │         │ True        │ ()      │ float64 │     1   │
├─────────────────────────────┼───────────┼─────────────┼─────────┼─────────────┼─────────┼─────────┼───

In [18]:
#gpflow.mean_functions.Zero())
m = gpflow.models.GPR((X_train, Y_train), kernel=k, mean_function=None)#gpflow.mean_functions.Zero())
m.likelihood.variance.assign(0.01)
#m.kernel.lengthscales.assign(np.array([0.3, 0.3, 0.3]))

<tf.Variable 'UnreadVariable' shape=() dtype=float64, numpy=-4.600266525158521>

In [19]:
opt = gpflow.optimizers.Scipy()

In [ ]:
opt_logs = opt.minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=5))
print_summary(m)